# get less than a month at a time

In [7]:
import requests
import pandas as pd
import sqlite3

In [2]:
start_date = "2024-01-01"
end_date = "2024-01-25"

df = pd.DataFrame(
    columns=[
        "period",
        "respondent",
        "respondent-name",
        "fueltype",
        "type-name",
        "value",
        "value-units",
    ]
)
# API endpoint
KEY = "jbCV5CVC8OBbXWyk1Vex3OLlQMIvqYQnmD0hcyAr"
url = f"https://api.eia.gov/v2/electricity/rto/fuel-type-data/data/?api_key={KEY}&frequency=hourly&data[0]=value&facets[fueltype][]=COL&facets[fueltype][]=NG&facets[fueltype][]=NUC&facets[fueltype][]=OIL&facets[fueltype][]=OTH&facets[fueltype][]=SUN&facets[fueltype][]=WAT&facets[fueltype][]=WND&facets[respondent][]=PJM&start={start_date}T00&end={end_date}T00&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000"
response = requests.get(url)
data = response.json()
tdf = pd.DataFrame.from_dict(data["response"]["data"])
df = pd.concat([df, tdf], ignore_index=True)
df = df.sort_values(by=["period"]).reset_index(drop=True)
df.drop(columns=["respondent-name"], inplace=True)
df.rename(columns={"period": "date"}, inplace=True)
df["date"] = pd.to_datetime(df["date"])

In [3]:
conn = sqlite3.connect("data/power_production.db")
df.to_sql("power_production", conn, if_exists="append", index=False)
conn.close()

In [8]:
conn = sqlite3.connect("data/power_production.db")
df = pd.read_sql("SELECT * FROM power_production", conn)
conn.close()

In [6]:
df['date'] = pd.to_datetime(df['date'])

In [9]:
df

,date,respondent,fueltype,type-name,value,value-units
0,2018-07-01 05:00:00,PJM,COL,Coal,34345.0,megawatthours
1,2018-07-01 05:00:00,PJM,OTH,Other,934.0,megawatthours
2,2018-07-01 05:00:00,PJM,NUC,Nuclear,33643.0,megawatthours
3,2018-07-01 05:00:00,PJM,WAT,Hydro,538.0,megawatthours
4,2018-07-01 05:00:00,PJM,OIL,Petroleum,183.0,megawatthours
...,...,...,...,...,...,...
329051,2024-01-25 00:00:00,PJM,OTH,Other,822.0,megawatthours
329052,2024-01-25 00:00:00,PJM,COL,Coal,18656.0,megawatthours
329053,2024-01-25 00:00:00,PJM,NUC,Nuclear,33186.0,megawatthours
329054,2024-01-25 00:00:00,PJM,WAT,Hydro,3061.0,megawatthours
